In [1]:
import logging
import sys

from llama_index.callbacks import CallbackManager, LlamaDebugHandler
from llama_index.llms import LlamaCPP
from llama_index.llms.llama_utils import messages_to_prompt, completion_to_prompt

logging.basicConfig(stream=sys.stdout, level=logging.INFO)  # Change INFO to DEBUG if you want more extensive logging
logging.getLogger().addHandler(logging.StreamHandler(stream=sys.stdout))

llama_debug = LlamaDebugHandler(print_trace_on_end=True)
callback_manager = CallbackManager([llama_debug])

In [2]:
llm = LlamaCPP(
    model_url="https://huggingface.co/TheBloke/Llama-2-13B-chat-GGUF/resolve/main/llama-2-13b-chat.Q4_K_M.gguf",
    
    # optionally, you can set the path to a pre-downloaded model instead of model_url
    model_path=None,
    
    temperature=0.0,
    max_new_tokens=1024,
    
    # llama2 has a context window of 4096 tokens, but we set it lower to allow for some wiggle room
    context_window=3900,  # note, this sets n_ctx in the model_kwargs below, so you don't need to pass it there.
    
    # kwargs to pass to __call__()
    generate_kwargs={},
    
    # kwargs to pass to __init__()
    # set to at least 1 to use GPU
    model_kwargs={"n_gpu_layers": 4}, # I need to play with this and see if it actually helps
    
    # transform inputs into Llama2 format
    messages_to_prompt=messages_to_prompt,
    completion_to_prompt=completion_to_prompt,
    verbose=True,
)

AVX = 1 | AVX2 = 1 | AVX512 = 0 | AVX512_VBMI = 0 | AVX512_VNNI = 0 | FMA = 1 | NEON = 0 | ARM_FMA = 0 | F16C = 1 | FP16_VA = 0 | WASM_SIMD = 0 | BLAS = 0 | SSE3 = 1 | SSSE3 = 0 | VSX = 0 | 


In [3]:
import os
from llama_index import VectorStoreIndex, SimpleDirectoryReader, ServiceContext

from llama_index.llms import LLMMetadata

transcript_directory = "./test"
storage_directory = "./result_indexing"

# Add filename as metadata to each chunk associated with a document/transcript
filename_fn = lambda filename: {'source': os.path.splitext(os.path.basename(filename))[0]}
documents = SimpleDirectoryReader(transcript_directory, filename_as_id=True, 
                                  file_metadata=filename_fn).load_data()

# Exclude metadata from the LLM, meaning it won't read it when generating a response.
# Future - consider looping over documents and setting the id_ to basename, instead of fullpath
[document.excluded_llm_metadata_keys.append('source') for document in documents]


# chunk_size - It defines the size of the chunks (or nodes) that documents are broken into when they are indexed by LlamaIndex
service_context = ServiceContext.from_defaults( chunk_size=1024,
                                               embed_model="local", 
                                               callback_manager=callback_manager,
                                               llm_predictor=llm,
                                               )

# Build the index
index = VectorStoreIndex.from_documents(documents, service_context=service_context, show_progress=True)

# Persist the index to disk
index.storage_context.persist(persist_dir=storage_directory)

c:\Users\asus\AppData\Local\Programs\Python\Python311\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


INFO:sentence_transformers.SentenceTransformer:Load pretrained SentenceTransformer: BAAI/bge-small-en
Load pretrained SentenceTransformer: BAAI/bge-small-en
INFO:sentence_transformers.SentenceTransformer:Use pytorch device: cpu
Use pytorch device: cpu


Generating embeddings: 100%|██████████| 5861/5861 [54:40<00:00,  1.79it/s]


**********
Trace: index_construction
    |_CBEventType.NODE_PARSING ->  16.343106 seconds
      |_CBEventType.CHUNKING ->  0.199091 seconds
      |_CBEventType.CHUNKING ->  0.25201 seconds
      |_CBEventType.CHUNKING ->  0.010078 seconds
      |_CBEventType.CHUNKING ->  0.357746 seconds
      |_CBEventType.CHUNKING ->  0.035521 seconds
      |_CBEventType.CHUNKING ->  0.023004 seconds
      |_CBEventType.CHUNKING ->  0.102532 seconds
      |_CBEventType.CHUNKING ->  0.086518 seconds
      |_CBEventType.CHUNKING ->  0.529762 seconds
      |_CBEventType.CHUNKING ->  0.104027 seconds
      |_CBEventType.CHUNKING ->  0.166611 seconds
      |_CBEventType.CHUNKING ->  0.007041 seconds
      |_CBEventType.CHUNKING ->  0.059566 seconds
      |_CBEventType.CHUNKING ->  0.309621 seconds
      |_CBEventType.CHUNKING ->  0.090519 seconds
      |_CBEventType.CHUNKING ->  0.200019 seconds
      |_CBEventType.CHUNKING ->  0.055619 seconds
      |_CBEventType.CHUNKING ->  0.113415 seconds
      |_CBE

In [4]:
# Now you can load the index from disk when needed, and not rebuild it each time.
from llama_index import VectorStoreIndex, SimpleDirectoryReader, ServiceContext
from llama_index import StorageContext, load_index_from_storage

transcript_directory = "./test"
storage_directory = "./result_indexing"

service_context = ServiceContext.from_defaults(llm=llm, chunk_size=1024,
                                               embed_model="local",
                                               callback_manager=callback_manager)

storage_context = StorageContext.from_defaults(persist_dir=storage_directory)
index = load_index_from_storage(storage_context, service_context=service_context)

INFO:sentence_transformers.SentenceTransformer:Load pretrained SentenceTransformer: BAAI/bge-small-en
Load pretrained SentenceTransformer: BAAI/bge-small-en
INFO:sentence_transformers.SentenceTransformer:Use pytorch device: cpu
Use pytorch device: cpu
INFO:llama_index.indices.loading:Loading all indices.
Loading all indices.
**********
Trace: index_construction
**********


In [27]:
from IPython.display import Markdown, display
from llama_index.prompts import PromptTemplate

query_engine = index.as_query_engine(service_context=service_context,
                                    #  streaming=True,
                                     similarity_top_k=1)

response = query_engine.query("Tampilkan tafsir surah Al-Baqarah ayat ke 2?")
# display(Markdown(f"<b>{response}</b>"))

Batches: 100%|██████████| 1/1 [00:00<00:00, 23.26it/s]
Llama.generate: prefix-match hit


**********
Trace: query
    |_CBEventType.QUERY ->  317.177359 seconds
      |_CBEventType.RETRIEVE ->  0.252029 seconds
        |_CBEventType.EMBEDDING ->  0.050986 seconds
      |_CBEventType.SYNTHESIZE ->  316.92533 seconds
        |_CBEventType.TEMPLATING ->  0.0 seconds
        |_CBEventType.LLM ->  316.915332 seconds
**********


In [32]:
#convert response to string
print(type(response))

response_str = str(response)
display(Markdown(f"<b>{response}</b>"))

<class 'llama_index.response.schema.Response'>


<b>  Based on the context information provided, I will provide the tafsir of Surah Al-Baqarah, Ayat 2.

Tafsir of Surah Al-Baqarah, Ayat 2:

Ayat 2 of Surah Al-Baqarah reads: "This is the Book about which there is no doubt, a guidance for those who are conscious of Allah." (Surah Al-Baqarah, verse 2)

The tafsir of this ayat is as follows:

Imam Malik and his followers, as well as Imam Abu Hanifah and his followers, hold the opinion that Basmalah is a separate ayat and should not be recited in salat. They base their argument on the hadith of Anas bin Malik, who reported that the Prophet (peace be upon him) and his companions, including Abu Bakr, Umar, and Usman, used to recite al-hamdulillahi rabbil 'alamin without saying Bismillahirrahmanirrahim at the beginning or end of their salat. (Narrated by Bukhari and Muslim)

On the other hand, Imam Syafi'i and his followers believe that Basmalah is part of al-Fatihah and should be recited with a loud voice in salat, just like other ayats. They base their argument on the hadith of Ibnu 'Abbas, who reported that the Prophet (peace be upon him) used to emphasize the recitation of Bismillahirrahmanirrahim. (Narrated by Hakim in al-Mustadrak and considered sahih)

Umm Salamah also reported that the Prophet (peace be upon him) used to pause and emphasize the recitation of Bismillahirrahmanirrahim, al-hamdulillahi Rabbil-'alamin, ar-Rahmanir-rahim, Maliki Yaumid-din. (Narrated by Ahmad, Abu Daud, Ibnu Khuzaimah, and Hakim, and considered sahih)

Abu Hurairah also reported that the Prophet (peace be upon him) used to recite Basmalah with a loud voice.

In conclusion, there are different opinions among the companions of the Prophet (peace be upon him) and the great imams of Islam regarding the recitation of Basmalah in salat. However, all of them agree that Basmalah is a powerful and important ayat that should be recited with respect and attention.</b>

In [33]:
from deep_translator import GoogleTranslator

# Use any translator you like, in this example GoogleTranslator
translated = GoogleTranslator(source='en', target='id').translate(response_str)  # output -> Weiter so, du bist großarti

In [34]:
display(Markdown(f"<b>{translated}</b>"))

<b>Berdasarkan konteks informasi yang diberikan, saya akan memberikan tafsir Surat Al-Baqarah Ayat 2.

Tafsir Surat Al-Baqarah Ayat 2 :

Ayat 2 Surat Al-Baqarah berbunyi: “Inilah Kitab yang tidak ada keraguannya, petunjuk bagi orang-orang yang bertakwa.” (Surah Al-Baqarah, ayat 2)

Tafsir ayat ini adalah sebagai berikut:

Imam Malik dan para pengikutnya, serta Imam Abu Hanifah dan para pengikutnya, berpendapat bahwa Basmalah adalah ayat tersendiri dan tidak boleh dibaca dalam shalat. Mereka mendasarkan dalilnya pada hadis Anas bin Malik, yang meriwayatkan bahwa Nabi SAW dan para sahabatnya, antara lain Abu Bakar, Umar, dan Usman, biasa membacakan al-hamdulillahi rabbil 'alamin tanpa mengucapkan Bismillahirrahmanirrahim di awal. awal atau akhir shalatnya. (HR Bukhari dan Muslim)

Di sisi lain, Imam Syafi'i dan para pengikutnya berpendapat bahwa Basmalah merupakan bagian dari al-Fatihah dan harus dibaca dengan suara nyaring dalam shalat, sama seperti ayat-ayat lainnya. Mereka mendasarkan argumentasinya pada hadits Ibnu 'Abbas yang meriwayatkan bahwa Nabi SAW biasa menekankan bacaan Bismillahirrahmanirrahim. (HR. Hakim dalam al-Mustadrak dan dianggap sahih)

Ummu Salamah juga meriwayatkan bahwa Nabi SAW biasa memberi jeda dan menekankan bacaan Bismillahirrahmanirrahim, al-hamdulillahi Rabbil-'alamin, ar-Rahmanir-rahim, Maliki Yaumid-din. (HR. Ahmad, Abu Daud, Ibnu Khuzaimah, dan Hakim, dan dianggap sahih)

Abu Hurairah juga meriwayatkan bahwa Nabi (saw) biasa membaca Basmalah dengan suara keras.

Kesimpulannya, terdapat perbedaan pendapat di kalangan sahabat Nabi (saw) dan para imam besar Islam mengenai pembacaan Basmalah dalam shalat. Namun, mereka semua sepakat bahwa Basmalah adalah ayat yang kuat dan penting yang harus dibaca dengan penuh hormat dan perhatian.</b>

In [37]:
# Print the number of source nodes
num_source_nodes = len(response.source_nodes)
print(f"Number of source nodes: {num_source_nodes}")

# Loop over source nodes and print meta data
for s in response.source_nodes:
    print(f"Node Score: {s.score}")
    print(s.node.metadata)

Number of source nodes: 1
Node Score: 0.8880539514818451
{'source': 'An-Nur_KEMENAG'}


# S3 integration

In [1]:
import boto3
import boto3.session
import botocore
from decouple import config
import s3fs
from os import getenv
import dotenv
import os
from llama_index import download_loader


In [2]:
dotenv.load_dotenv(".env")

True

In [3]:
AWS_ACCESS_KEY_ID = os.environ['AWS_ACCESS_KEY_ID']
AWS_SECRET_ACCESS_KEY = os.environ['AWS_SECRET_ACCESS_KEY']
AWS_BUCKET = os.environ['AWS_BUCKET']
AWS_ENDPOINT = os.environ['AWS_ENDPOINT']
AWS_FOLDER = os.environ['AWS_FOLDER']

In [ ]:
S3Reader = download_loader("S3Reader")

loader = S3Reader(bucket='klikgeni', key='source_alquran/', aws_access_id='[ACCESS_KEY_ID]', aws_access_secret='[ACCESS_KEY_SECRET]')
documents = loader.load_data()

In [8]:
OpendalS3Reader = download_loader("OpendalS3Reader")

loader = OpendalS3Reader(
    bucket=AWS_BUCKET,
    path='{AWS_BUCKET}/{AWS_FOLDER}',
    access_key_id=AWS_ACCESS_KEY_ID,
    secret_access_key=AWS_SECRET_ACCESS_KEY,
)
documents = loader.load_data()

RuntimeError: asyncio.run() cannot be called from a running event loop

In [7]:

assert AWS_ACCESS_KEY_ID is not None and AWS_ACCESS_KEY_ID != ""

s3 = s3fs.S3FileSystem(
   key=AWS_ACCESS_KEY_ID,
   secret=AWS_SECRET_ACCESS_KEY,
   endpoint_url=AWS_ENDPOINT,
)

# save index to remote blob storage
index.set_index_id("vector_index")
# this is {bucket_name}/{index_name}
index.storage_context.persist('klikgeni/index_quran', fs=s3)
index.service_context.persist()

In [10]:
# load index from s3
sc = StorageContext.from_defaults(persist_dir='klikgeni/index_quran', fs=s3)
index2 = load_index_from_storage(sc, 'vector_index', service_context=service_context)

No existing llama_index.graph_stores.simple found at klikgeni/index_quran\graph_store.json. Initializing a new graph_store from scratch. 
INFO:llama_index.indices.loading:Loading indices with ids: ['vector_index']
Loading indices with ids: ['vector_index']
**********
Trace: index_construction
**********


In [16]:
from IPython.display import Markdown, display
from llama_index.prompts import PromptTemplate

query_engine_2 = index2.as_query_engine(service_context=service_context,
                                    #  streaming=True,
                                     similarity_top_k=1)

response = query_engine_2.query("Apa dalil untuk hukum namimah?")
# display(Markdown(f"<b>{response}</b>"))

Batches: 100%|██████████| 1/1 [00:00<00:00,  2.79it/s]
Llama.generate: prefix-match hit


**********
Trace: query
    |_CBEventType.QUERY ->  306.467679 seconds
      |_CBEventType.RETRIEVE ->  0.443534 seconds
        |_CBEventType.EMBEDDING ->  0.409545 seconds
      |_CBEventType.SYNTHESIZE ->  306.024145 seconds
        |_CBEventType.TEMPLATING ->  0.0 seconds
        |_CBEventType.LLM ->  305.992667 seconds
**********


In [17]:
display(Markdown(f"<b>{response}</b>"))

<b>  Based on the context information provided, the term "namimah" refers to a situation where one person intentionally speaks ill or spreads false rumors about another person with the intention of damaging their reputation or relationships.

According to Imam An-Nawawi, as quoted in the reference text, namimah is defined as "menyampaikan perkataan seseorang kepada orang lain dengan tujuan merusak hubungan di antara mereka" (Syarh al-Nawawi ‘ala Shahih Muslim, 1/214).

In Islamic law, namimah is considered a form of slander and is prohibited. The Prophet Muhammad (peace be upon him) said, "Whoever intentionally lies about another person, let him take a place in the Hellfire" (Narrated by Bukhari and Muslim).

Therefore, the hukum (ruling) for namimah is that it is haram (forbidden) and can lead to severe consequences in the hereafter. It is important to be mindful of our words and actions, and to refrain from intentionally harming others through false speech or rumors.</b>